In [1]:
import os
import json
import pickle
import pandas as pd
import numpy as np
import tqdm

In [3]:
path_result = "/home/coder/MixPL/work_dirs/mixpl_dino-qixiongjiye"
pkl_file = "yh_qixiongjiye@100-mixpl_dino-iter60k.pkl"
with open(os.path.join(path_result, pkl_file), 'rb') as f:
    results = pickle.load(f)
    
dir_save = os.path.join(path_result.replace(pkl_file, ""), "infer")
os.makedirs(dir_save, exist_ok=True)

In [4]:
def deal_one(info):
    name, _ = os.path.splitext(os.path.basename(info["img_path"]))
    shape = info["ori_shape"]
    bboxes = info["pred_instances"]["bboxes"]
    scores = info["pred_instances"]["scores"]
    labels = info["pred_instances"]["labels"]
    texts = []
    for bbox, score, label in zip(bboxes.tolist(), scores.tolist(), labels.tolist()):
        bbox = np.array(bbox).reshape(2, 2)
        center = (bbox[1] + bbox[0]) / 2 / np.array(shape)
        wh = (bbox[1] - bbox[0]) / np.array(shape)
        text = f"{label} {center[0]} {center[1]} {wh[0]} {wh[1]} {score}"
        texts.append(text)
    path_save = os.path.join(dir_save, f"{name}.txt")
    with open(path_save, 'w') as f:
        f.write("\n".join(texts))
        f.write("\n")
    return texts

In [5]:
for info in tqdm.tqdm(results):
    deal_one(info)

100%|██████████| 13533/13533 [01:21<00:00, 166.75it/s]


In [6]:
from mmengine import Config

In [16]:
path_cfg = os.path.join(path_result, path_result.split('/')[-1] + '.py')
cfg = Config.fromfile(path_cfg)
with open(path_cfg.replace(".py", '_cfg.json'), 'w') as f:
    json.dump(cfg.to_dict(), f, indent=2)

In [12]:
!python /home/coder/downstreamtask-evaluation/downstreamtask_evaluation/det/convert_labels_flexible.py \
    --pred_dir "/home/coder/MixPL/work_dirs/mixpl_dino-qixiongjiye/infer" \
    --label_dir "/yinghepool/HT_data/temp/qj2/train_data_png_filter60/QiXiongJiYe/labels/val" \
    --output "/home/coder/MixPL/work_dirs/mixpl_dino-qixiongjiye/infer.json" \
    --dataset "qixiong_jiye"

标签格式转换脚本
预测框路径: /home/coder/MixPL/work_dirs/mixpl_dino-qixiongjiye/infer
金标准路径: /yinghepool/HT_data/temp/qj2/train_data_png_filter60/QiXiongJiYe/labels/val
输出文件: /home/coder/MixPL/work_dirs/mixpl_dino-qixiongjiye/infer.json
数据集名称: qixiong_jiye
发现 13533 个金标准标签文件
已处理 50 个文件...
已处理 100 个文件...
已处理 150 个文件...
已处理 200 个文件...
已处理 250 个文件...
已处理 300 个文件...
已处理 350 个文件...
已处理 400 个文件...
已处理 450 个文件...
已处理 500 个文件...
已处理 550 个文件...
已处理 600 个文件...
已处理 650 个文件...
已处理 700 个文件...
已处理 750 个文件...
已处理 800 个文件...
已处理 850 个文件...
已处理 900 个文件...
已处理 950 个文件...
已处理 1000 个文件...
已处理 1050 个文件...
已处理 1100 个文件...
已处理 1150 个文件...
已处理 1200 个文件...
已处理 1250 个文件...
已处理 1300 个文件...
已处理 1350 个文件...
已处理 1400 个文件...
已处理 1450 个文件...
已处理 1500 个文件...
已处理 1550 个文件...
已处理 1600 个文件...
已处理 1650 个文件...
已处理 1700 个文件...
已处理 1750 个文件...
已处理 1800 个文件...
已处理 1850 个文件...
已处理 1900 个文件...
已处理 1950 个文件...
已处理 2000 个文件...
已处理 2050 个文件...
已处理 2100 个文件...
已处理 2150 个文件...
已处理 2200 个文件...
已处理 2250 个文件...
已处理 2300 个文件...
已处理 2350 个文件...
已处理 24

In [7]:
# !python /yinghepool/zhangshuheng/codes/downstreamtask-evaluation/detection/convert_labels_flexible.py \
#     --pred_dir "/yinghepool/zhangshuheng/codes/mmdetection/work_dirs/grounding_dino_swin-t_finetune_8xb4_50e_qixiong_jiye_linchuang/qixiong_jiye_linchuang-7ep/val" \
#     --label_dir "/yinghepool/HT_data/temp/qj_test_005-1-20250718145022/train_data_png_filter60/QiXiongJiYe/labels/val" \
#     --output "/yinghepool/zhangshuheng/codes/mmdetection/work_dirs/grounding_dino_swin-t_finetune_8xb4_50e_qixiong_jiye_linchuang/qixiong_jiye_linchuang-7ep/val.json" \
#     --dataset "qixiong_jiye_linchuang"

In [17]:
!python /yinghepool/zhangshuheng/codes/downstreamtask-evaluation/evaluate.py \
    -i "/home/coder/MixPL/work_dirs/mixpl_dino-qixiongjiye/infer.json" \
    --detect \
    --model "/home/coder/MixPL/work_dirs/mixpl_dino-qixiongjiye/iter_60000.pth" \
    --config "/home/coder/MixPL/work_dirs/mixpl_dino-qixiongjiye/mixpl_dino-qixiongjiye_cfg.json" \
    -conf 0.5 \
    --save_path "/home/coder/MixPL/work_dirs/mixpl_dino-qixiongjiye/standard_infer_output" \
    --verbose

2025-08-06 18:45:46 - __main__ - INFO - 日志文件保存到: /home/coder/MixPL/work_dirs/mixpl_dino-qixiongjiye/standard_infer_output/evaluation_20250806_184546.log
2025-08-06 18:45:46 - __main__ - INFO - 开始模型评估...
2025-08-06 18:45:56 - downstreamtask_evaluation.detection - INFO - 日志文件保存到: /home/coder/MixPL/work_dirs/mixpl_dino-qixiongjiye/standard_infer_output/evaluation_20250806_184556.log
2025-08-06 18:45:56 - downstreamtask_evaluation.detection - INFO - 开始评估检测模型... (指定置信度模式)
2025-08-06 18:45:56 - downstreamtask_evaluation.detection - INFO - 🎯 模式: 使用指定置信度阈值 0.5
2025-08-06 18:45:56 - downstreamtask_evaluation.detection - DEBUG - 模型路径: /home/coder/MixPL/work_dirs/mixpl_dino-qixiongjiye/iter_60000.pth
2025-08-06 18:45:56 - downstreamtask_evaluation.detection - INFO - 数据长度: 13533
2025-08-06 18:45:56 - downstreamtask_evaluation.detection - INFO - IoU阈值范围: 0.1 - 0.95
Processing evaluate: 100%|███████████████| 13533/13533 [00:20<00:00, 672.39it/s]
2025-08-06 18:46:19 - downstreamtask_evaluation.detect